# SQLAlchemy ORM Reflection with Joins

### Here we "reflect" (i.e inspect/read) the design of an existing database.

We read what is out there.  

This of course is awesome for existing databases.

This keeps you from having to manually create classes

In [1]:
# dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin

In [2]:
# connect to database
db_path = "corgies.db"
engine = create_engine(f"sqlite:///{db_path}")
conn = engine.connect()

## What if I just want to know what's out there?

I don't want to update anything, I just want to be able to see the design

In [3]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [4]:
tables = inspector.get_table_names()
print(tables)

['grades', 'pets', 'training']


In [5]:
for table in tables:
    print("\n")
    print('-' * 12)
    print(f"table '{table}' has the following columns:")
    print('-' * 12)
    for column in inspector.get_columns(table):
        print(f"name: {column['name']}   column type: {column['type']}")
        



------------
table 'grades' has the following columns:
------------
name: pet_id   column type: INTEGER
name: task_id   column type: INTEGER
name: grade   column type: TEXT
name: comments   column type: TEXT


------------
table 'pets' has the following columns:
------------
name: id   column type: INTEGER
name: name   column type: TEXT
name: age   column type: INTEGER
name: breed   column type: TEXT
name: age_adopted   column type: TEXT


------------
table 'training' has the following columns:
------------
name: id   column type: INTEGER
name: task   column type: TEXT
name: description   column type: TEXT


## I NEED MORE POWER !!!!

Things have got to change

In [6]:
# let's create the base 
Base = automap_base()

In [7]:
# let's find out what's in this darn database !
Base.prepare(engine, reflect=True)

In [8]:
tables = Base.classes.keys()
tables

['grades', 'pets', 'training']

### Classes = Tables
Remember classes are representations of tables in our database.  

So by getting the classe names, we are getting the table names

In [9]:
Grades = Base.classes['grades']
Pets = Base.classes['pets']
Training = Base.classes['training']

In [10]:
# remember we need a session for ORM
session = Session(engine)

In [11]:
# idea base from: https://riptutorial.com/sqlalchemy/example/6614/converting-a-query-result-to-dict
def object_as_dict(obj):
    """
    This function takes in a Class instance and converts it to a dictionary
    """
    obj_count = 1
    try:
        obj_count = len(obj)
    except:
        pass
    if  obj_count == 1:
        base_dict = {c.key: getattr(obj, c.key)
            for c in inspect(obj).mapper.column_attrs}
        return base_dict
    else:
        cur_obj = obj[0]
        base_dict = {c.key: getattr(cur_obj, c.key) for c in inspect(cur_obj).mapper.column_attrs}
        for i in range(1, obj_count):
            cur_obj = obj[i]
            cur_dict = {c.key: getattr(cur_obj, c.key) for c in inspect(cur_obj).mapper.column_attrs}
            base_dict = {**base_dict, **cur_dict} 
        return base_dict    

In [12]:
# from jeff LOL
def query_to_list_of_dicts(cur_query):
    """
    From a query object return a list of dictionaries
    """
    return [object_as_dict(row) for row in cur_query]

In [13]:
pets = query_to_list_of_dicts(session.query(Pets))
# which can easily be made into a dataframe!!!!
pd.DataFrame(pets)

,id,name,age,breed,age_adopted
0,1,Patterson,11,Corgi,8 weeks
1,2,Judson,10,Corgi,3 years
2,3,Emily,?,Corgi,None


In [14]:
training = query_to_list_of_dicts(session.query(Training))
# for display purposes only
pd.DataFrame(training)

,id,task,description
0,1,sit,sit with bottom on floor
1,2,I feel faint,"lay flat on back legs up like a ""faint"""
2,3,roll over,completely roll over
3,4,speak,bark loud and clear
4,5,lay down,lay with all four legs relaxed


In [15]:
grades = query_to_list_of_dicts(session.query(Grades))
# for display purposes only
pd.DataFrame(grades)

,pet_id,task_id,grade,comments
0,1,1,A,good
1,1,2,A-,"quick but needed ""do a good one"""
2,1,3,B,weak attempt
3,1,4,B,quiet
4,1,5,C,reluctant
5,2,1,B,required butt bump
6,2,2,B-,more of a swoon
7,2,4,A,loud and deep
8,2,5,A,good


# Basic Join

## Engine Inner Join

In [16]:
inner_join_sql = """
select p.*, g.* 
from pets p 
join grades g on p.id = g.pet_id
"""
engine.execute(inner_join_sql).fetchall()

[(1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 1, 'A', 'good'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 2, 'A-', 'quick but needed "do a good one"'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 3, 'B', 'weak attempt'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 4, 'B', 'quiet'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 5, 'C', 'reluctant'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 1, 'B', 'required butt bump'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 2, 'B-', 'more of a swoon'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 4, 'A', 'loud and deep'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 5, 'A', 'good')]

## ORM Inner Join

In [17]:
results = session.query(Pets, Grades).filter(Pets.id == Grades.pet_id)
pd.DataFrame(query_to_list_of_dicts(results))

,id,name,age,breed,age_adopted,pet_id,task_id,grade,comments
0,1,Patterson,11,Corgi,8 weeks,1,1,A,good
1,1,Patterson,11,Corgi,8 weeks,1,2,A-,"quick but needed ""do a good one"""
2,1,Patterson,11,Corgi,8 weeks,1,3,B,weak attempt
3,1,Patterson,11,Corgi,8 weeks,1,4,B,quiet
4,1,Patterson,11,Corgi,8 weeks,1,5,C,reluctant
5,2,Judson,10,Corgi,3 years,2,1,B,required butt bump
6,2,Judson,10,Corgi,3 years,2,2,B-,more of a swoon
7,2,Judson,10,Corgi,3 years,2,4,A,loud and deep
8,2,Judson,10,Corgi,3 years,2,5,A,good


# Outer Joins

## Engine Version

Note the "None" values in the cells

In [18]:
outer_join_sql = """
select p.*, g.* 
from pets p 
left join grades g on p.id = g.pet_id
"""
engine.execute(outer_join_sql).fetchall()

[(1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 1, 'A', 'good'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 2, 'A-', 'quick but needed "do a good one"'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 3, 'B', 'weak attempt'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 4, 'B', 'quiet'),
 (1, 'Patterson', 11, 'Corgi', '8 weeks', 1, 5, 'C', 'reluctant'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 1, 'B', 'required butt bump'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 2, 'B-', 'more of a swoon'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 4, 'A', 'loud and deep'),
 (2, 'Judson', 10, 'Corgi', '3 years', 2, 5, 'A', 'good'),
 (3, 'Emily', '?', 'Corgi', None, None, None, None, None)]

## ORM Outer Join

Note how the "None" type creates challenges that must be addressed.

Currently the query_to_list_of_dicts function breaks upon None types.

In [19]:
results = session.query(Pets, Grades).outerjoin(Grades, Pets.id == Grades.pet_id).all()
for pet, grade in results:
    if grade != None:
        print(f"Pet: {pet.name} received an {grade.grade} on task: {grade.task_id}")
    else:
        print(f"Pet: {pet.name} has taken no training to be graded")


Pet: Patterson received an A on task: 1
Pet: Patterson received an A- on task: 2
Pet: Patterson received an B on task: 3
Pet: Patterson received an B on task: 4
Pet: Patterson received an C on task: 5
Pet: Judson received an B on task: 1
Pet: Judson received an B- on task: 2
Pet: Judson received an A on task: 4
Pet: Judson received an A on task: 5
Pet: Emily has taken no training to be graded


In [20]:
# always clean up after yourself
session.close()